In [1]:
from datetime import datetime
from fractions import Fraction

import configparser
import pandas as pd

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')
last_paid = datetime.strptime(config['DEFAULT']['last_paid'], '%Y-%m-%d')
mint_analysis = datetime.strptime(config['DEFAULT']['mint_analysis'], '%Y-%m-%d')

In [3]:
chase = pd.read_csv('ChaseCC.csv')
citi = pd.read_csv('CitiCC.csv')
mint = pd.read_csv('Mint.csv')

In [4]:
chase['trans_date'] = pd.to_datetime(chase['Transaction Date'])
citi['trans_date'] = pd.to_datetime(citi['Date'])
mint['trans_date'] = pd.to_datetime(mint['Date'])

In [5]:
chase = chase[chase['trans_date'] > last_paid]
chase = chase[chase['Type'] != 'Payment']
citi = citi[citi['trans_date'] > last_paid]
citi = citi[~citi['Description'].str.contains('PAYMENT, THANK YOU')]
mint = mint[mint['trans_date'] > mint_analysis]

In [6]:
chase['amt'] = chase['Amount'] * -1

In [7]:
citi.loc[(citi['Debit'].notnull()), 'amt'] = citi['Debit']
citi.loc[(citi['Credit'].notnull()), 'amt'] = citi['Credit']

In [8]:
mint.loc[(mint['Transaction Type'] == 'debit'), 'amt'] = mint['Amount']
mint.loc[(mint['Transaction Type'] == 'credit'), 'amt'] = mint['Amount'] * -1

In [9]:
chase = chase[['trans_date', 'amt', 'Description']]
citi = citi[['trans_date', 'amt', 'Description']]
mint = mint[['trans_date', 'amt', 'Category', 'Description', 'Original Description']]

In [10]:
chase.head(5)

,trans_date,amt,Description
0,2020-12-21,47.69,UBER EATS
1,2020-12-16,25.00,TVY*FITNESSYOURWAY
2,2020-12-14,32.66,UBER EATS
3,2020-12-13,42.77,UBER EATS
4,2020-12-10,114.54,SPRINT *WIRELESS


In [11]:
citi.head(5)

,trans_date,amt,Description
0,2020-12-24,-449.99,PAYPAL *B H PHOTO 4029357733 NY Digital Accoun...
1,2020-12-24,6.36,PAYPAL *AMAGICOM AB 35314369001 SWE
2,2020-12-22,115.62,TRADER JOE'S #692 QPS BRENTWOOD MO
3,2020-12-21,10.69,PAYPAL *SPOTIFYUSAI 4029357733 NY Digital Acco...
4,2020-12-19,126.14,Amazon Prime*YG8V21593 8665572820 WA


In [12]:
mint.head(5)

,trans_date,amt,Category,Description,Original Description
0,2020-12-28,115.24,Groceries,Trader Joe's,TRADER JOE'S # 692 BRENTWOOD MO
1,2020-12-28,262.66,Credit Card Payment,CHASE CREDIT CRD,ACH DEBIT CHASE CREDIT CRD
2,2020-12-28,1550.96,Credit Card Payment,CITI CARD ONLINE,ACH DEBIT CITI CARD ONLINE
3,2020-12-27,12.48,Restaurants,Uber Eats,UBER *EATS
4,2020-12-27,32.68,Restaurants,Uber.com,UBER * EATS PENDING


In [13]:
round(citi['amt'].sum(),2)

1492.11

In [14]:
income_bk = round(float(config['BUDGET']['income_bk_raw']) * float(Fraction(config['BUDGET']['income_bk_mult'])),2)
income_bk

4855.95

In [15]:
categories = pd.read_excel('budget_categories.xlsx', sheet_name='main', engine='openpyxl')
categories.sort_values(by=['sort1', 'sort2', 'sort3'], inplace=True)
categories.reset_index(drop=True, inplace=True)
categories.head(5)

,sort1,sort2,sort3,category,sub_group,group,show_cat,show_sub,show_group,recurrence,recur_date,necessity
0,1,0,0,Income,Income,Income,False,True,True,NaN,NaT,Ignore
1,1,1,1,Paycheck,Income,Income,True,True,True,bi-weekly,2021-01-07,Ignore
2,1,1,2,Bonus,Income,Income,False,True,True,NaN,NaT,Ignore
3,1,1,3,Interest Income,Income,Income,False,True,True,NaN,NaT,Ignore
4,1,1,4,Rental Income,Income,Income,False,True,True,NaN,NaT,Ignore


In [16]:
mint = mint.groupby(by=['Category']).sum()
mint.head(5)

,amt
Category,
ATM Fee,18.50
Air Travel,59.65
Alcohol & Bars,1999.75
Auto Insurance,362.66
Credit Card Payment,-3969.78


In [17]:
budget_temp = categories.merge(mint, how='left', left_on='category', right_on='Category')
budget_temp.head(5)

,sort1,sort2,sort3,category,sub_group,group,show_cat,show_sub,show_group,recurrence,recur_date,necessity,amt
0,1,0,0,Income,Income,Income,False,True,True,NaN,NaT,Ignore,-296.08
1,1,1,1,Paycheck,Income,Income,True,True,True,bi-weekly,2021-01-07,Ignore,-98589.79
2,1,1,2,Bonus,Income,Income,False,True,True,NaN,NaT,Ignore,NaN
3,1,1,3,Interest Income,Income,Income,False,True,True,NaN,NaT,Ignore,-1301.52
4,1,1,4,Rental Income,Income,Income,False,True,True,NaN,NaT,Ignore,NaN


In [18]:
months_mint = ((datetime.today() - mint_analysis).days)/(365.25/12)
budget_temp['amt'] = round(budget_temp['amt']/months_mint,2)

In [19]:
budget_temp['amt2'] = budget_temp.groupby(['sub_group'])['amt'].transform('sum')
budget_temp['amt3'] = budget_temp.groupby(['group'])['amt'].transform('sum')
budget_temp.sort_values(by=['sort1', 'sort2', 'sort3'], inplace=True)
budget_temp.head(5)

,sort1,sort2,sort3,category,sub_group,group,show_cat,show_sub,show_group,recurrence,recur_date,necessity,amt,amt2,amt3
0,1,0,0,Income,Income,Income,False,True,True,NaN,NaT,Ignore,-12.35,-4177.34,-4177.34
1,1,1,1,Paycheck,Income,Income,True,True,True,bi-weekly,2021-01-07,Ignore,-4110.72,-4177.34,-4177.34
2,1,1,2,Bonus,Income,Income,False,True,True,NaN,NaT,Ignore,NaN,-4177.34,-4177.34
3,1,1,3,Interest Income,Income,Income,False,True,True,NaN,NaT,Ignore,-54.27,-4177.34,-4177.34
4,1,1,4,Rental Income,Income,Income,False,True,True,NaN,NaT,Ignore,NaN,-4177.34,-4177.34


In [31]:
budget_temp.loc[9, ['category', 'amt']]

category    Home Insurance
amt                    NaN
Name: 9, dtype: object

In [32]:
budget_analysis = pd.DataFrame(columns = ['category', 'amt'])
prev_group = ''
prev_sub = ''
prev_cat = ''
count = 0
for index, row in budget_temp.iterrows():
    next_value = budget_temp.loc[index, 'group']
    if next_value == prev_group or budget_temp.loc[index, 'show_group'] == False:
        pass
    else:
        budget_analysis.loc[count, 'category'] = next_value
        budget_analysis.loc[count, 'amt'] = budget_temp.loc[index, 'amt3']
        prev_group = next_value
        count += 1
    next_value = budget_temp.loc[index, 'sub_group']
    if next_value == prev_group or next_value == prev_sub or budget_temp.loc[index, 'show_sub'] == False:
        pass
    else:
        budget_analysis.loc[count, 'category'] = next_value
        budget_analysis.loc[count, 'amt'] = budget_temp.loc[index, 'amt2']
        prev_sub = next_value
        count += 1
    next_value = budget_temp.loc[index, 'category']
    if next_value == prev_group or next_value == prev_sub or next_value == prev_cat or budget_temp.loc[index, 'show_cat'] == False:
        pass
    else:
        budget_analysis.loc[count, 'category'] = next_value
        budget_analysis.loc[count, 'amt'] = budget_temp.loc[index, 'amt']
        prev_cat = next_value
        count += 1

In [33]:
budget_analysis.to_csv('test.csv')

# To-Do's
- Store budget information
- Create dictionary of Mint categories to more general categories
- Create calendar of how to predict upcoming paychecks
- Make list/calendar of upcoming expenses
- Calculate remaining budget based off upcoming expenses
- Create budget analysis (regression and past month)
- Create graphs and visuals
- Machine Learning budget